In [ ]:
import  numpynumpy  as  npnp
import  matplotlib.pyplot  as plt
import matplotlib.image as mpimg

from models import Net

In [ ]:
import cv2
# load in color image for face detection
image = cv2.imread('images/obamas.jpg')


image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# plot the image
fig = plt.figure(figsize=(9,9))
plt.imshow(image)

In [ ]:
face_cascade = cv2.CascadeClassifier('detector_architectures\\haarcascade_frontalface_default.xml')


faces = face_cascade.detectMultiScale(image, 1.2, 2)


image_with_detections = image.copy()


for (x,y,w,h) in faces:
    # draw a rectangle around each detected face
    # you may also need to change the width of the rectangle drawn depending on image resolution
    cv2.rectangle(image_with_detections,(x,y),(x+w,y+h),(255,0,0),3) 

fig = plt.figure(figsize=(9,9))

plt.imshow(image_with_detections);

In [ ]:
net = gpu(Net())

net.load_state_dict(torch.load('keypoints_model_1.pt'))

net.eval()

In [ ]:
def visualize_output(faces, test_outputs):  
    batch_size = len(faces)
    for i, face in enumerate(faces):
        plt.figure(figsize=(8, 8))
        ax = plt.subplot(1, batch_size, i+1)

        # un-transform the predicted key_pts data
        predicted_key_pts = test_outputs[i].data
        predicted_key_pts = predicted_key_pts.numpy()
        # undo normalization of keypoints  
        predicted_key_pts = predicted_key_pts*50.0+100

        plt.imshow(face, cmap='gray')
        plt.scatter(predicted_key_pts[:, 0], predicted_key_pts[:, 1], s=20, marker='.', c='r')
        
        plt.axis('off')

    plt.show()

In [ ]:
from torch.autograd import Variable

image_copy = np.copy(image)
images, keypoints = [], []
PADDING = 25

# loop over the detected faces from your haar cascade
for (x,y,w,h) in faces:
    
    # Select the region of interest that is the face in the image 
    roi = image_copy[y-PADDING:y+h+PADDING, x-PADDING:x+w+PADDING]
    
    ## TODO: Convert the face region from RGB to grayscale
    roi = cv2.cvtColor(roi, cv2.COLOR_RGB2GRAY)

    ## TODO: Normalize the grayscale image so that its color range falls in [0,1] instead of [0,255]
    roi = (roi / 255.).astype(np.float32)
    
    ## TODO: Rescale the detected face to be the expected square size for your CNN (224x224, suggested)
    roi = cv2.resize(roi, (224, 224))
    images.append(roi)
    
    ## TODO: Reshape the numpy image shape (H x W x C) into a torch image shape (C x H x W)
    if len(roi.shape) == 2:
        roi = np.expand_dims(roi, axis=0)
    else:
        roi = np.rollaxis(roi, 2, 0)
    
    # Make it a batch of length 1
    # If you don't, this happens:
    # - https://discuss.pytorch.org/t/expected-stride-to-be-a-single-integer-value-or-a-list-of-1-values-to-match-the-convolution-dimensions-but-got-stride-1-1/17140
    # - https://stackoverflow.com/questions/50073358/pytorch-cnn-stride-error
    roi = np.expand_dims(roi, axis=0)
    
    ## TODO: Make facial keypoint predictions using your loaded, trained network 
    ## wrap each face region in a Variable and perform a forward pass to get the predicted facial keypoints
    roi = gpu(torch.from_numpy(roi).type(torch.FloatTensor))
    results = net.forward(roi)
    
    results = results.view(results.size()[0], 68, -1).cpu()
    keypoints.append(results[0])
    
## TODO: Display each detected face and the corresponding keypoints
visualize_output(images, keypoints)
